In [2]:
from bs4 import BeautifulSoup as bs
import requests

#### Task 1 : Get Info Box

In [3]:
#r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

#Convert to a beautiful soup object
#soup = bs(r.content)

#Print the HTML
#contents = soup.prettify()

In [4]:
#info_box = soup.find(class_='infobox vevent')
#info_rows = info_box.find_all('tr')
#for row in info_rows:
#    print(row.prettify())

In [5]:

# def get_content_value(row_data):
#    if row_data.find("li"):
#        return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row_data.find_all("li")]
#    else:
#        return row_data.get_text(" ",strip=True).replace("\xa0"," ")
#
#movie_info = {}
#for index, row in enumerate(info_rows):
#    if index == 0:
#        movie_info['Title'] = row.find("th").get_text(" ",strip=True)
#    elif index == 1:
#        continue
#    else:
#        content_key = row.find('th').get_text(" ",strip=True)
#        content_value = get_content_value(row.find('td'))
#        movie_info[content_key] = content_value
#
#movie_info 

#### Task 2 : Get Info Box for All Movies

In [6]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ",strip=True).replace("\xa0"," ")

def clean_tags(soup):
    for tag in soup.find_all(["sup","span"]):
        tag.decompose()

def get_info_box(url):

    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_='infobox vevent')
    info_rows = info_box.find_all('tr')
    
    clean_tags(soup)
    
    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['Title'] = row.find("th").get_text(" ",strip=True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find('th').get_text(" ",strip=True)
                content_value = get_content_value(row.find('td'))
                movie_info[content_key] = content_value
    return movie_info



In [7]:
get_info_box("https://en.wikipedia.org/wiki/One_Little_Indian_(film)")

{'Title': 'One Little Indian',
 'Directed by': 'Bernard McEveety',
 'Produced by': 'Winston Hibler',
 'Written by': 'Harry Spalding',
 'Starring': ['James Garner',
  'Vera Miles',
  'Pat Hingle',
  'Morgan Woodward',
  'Jodie Foster'],
 'Music by': 'Jerry Goldsmith',
 'Cinematography': 'Charles F. Wheeler',
 'Edited by': 'Robert Stafford',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['June 20, 1973'],
 'Running time': '90 Minutes',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$2 million'}

In [8]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org/"

movie_info_list = []
for index, movie in enumerate(movies):
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']

        movie_info_list.append(get_info_box(full_path))
    except Exception as e:
        print(movie.get_text())
        print(e)


Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
True-Life Adventures
'NoneType' object has no attribute 'find_all'


In [9]:
len(movie_info_list)

443

#### Save/Reload Moive Data

In [10]:
import json

def save_data(title,data):
    with open(title,'w',encoding='utf-8') as f:
        json.dump(data, f,ensure_ascii=False,indent=2)

def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)
        

In [11]:
save_data("disney_data_clean.json",movie_info_list)

#### Task 3 : Clean the Data

In [12]:
movie_info_list = load_data("disney_data_clean.json")

#### Subtasks
- Clean up references [1]   DONE!!
- Convert running time into an integer DONE!!
- Convert dates into datetime object
- Split up the long strings DONE!!
- Convert Budget & Box office to numbers DONE!!

In [13]:
# Clean up refernces (remove [1] [2] etc.)
# Done clean_tags()

In [14]:
# Split up the long strings 
#  header = row.find('th')
#           if header:

In [15]:
# Convert running time to integer 
# minute_to_integer()

In [16]:
#print([movie.get('Running time','N/A') for movie in movie_info_list])

In [17]:
def minute_to_integer(running_time):
    if running_time == "N/A":
        return None
    if isinstance(running_time, list):
        return int(running_time[0].split(" ")[0])
    else:
        return int(running_time.split(" ")[0])
    

for movie in movie_info_list:
    movie['Running time (int)'] = minute_to_integer(movie.get('Running time','N/A'))

In [18]:
#print([movie.get('Running time (int)','N/A') for movie in movie_info_list])

In [19]:
#print([movie.get("Title") for movie in movie_info_list])

In [20]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"
value_re = rf"\${number}"
word_re = rf"\${number}(-|\sto\s|-)?({number})?\s({amounts})"#\s => space

In [21]:
def parse_value_syntax(string):
    value_string = re.search(number,string).group()
    value = float(value_string.replace(",",""))
    return value

In [22]:
def word_to_value(word):
    value_dict = {"thousand": 1000,"million":1000000,"billion":1000000000}
    return value_dict[word]

In [23]:
def parse_word_syntax(string):
    value_string = re.search(number,string).group()
    value = float(value_string.replace(',',""))
    word = re.search(amounts,string,flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value

In [24]:
def money_conversion(money):
    if money == "N/A":
        return None
    if isinstance(money,list):
        money = money[0]

    word_sytnax = re.search(word_re,money,flags=re.I)
    value_syntax = re.search(value_re,money) 

    if word_sytnax:
        return parse_word_syntax(word_sytnax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())

In [25]:
print(money_conversion("$790 Millions"))
money_conversion(movie_info_list[-40]['Box office'])

790000000.0


133400000.0

In [26]:
for movie in movie_info_list:
    movie['Budget (float)'] = money_conversion(movie.get('Budget','N/A'))
    movie['Box office (float)'] = money_conversion(movie.get('Box office','N/A'))

In [27]:
movie_info_list[1]

{'Title': 'Snow White and the Seven Dwarfs',
 'Directed by': ['David Hand (supervising)',
  'William Cottrell',
  'Wilfred Jackson',
  'Larry Morey',
  'Perce Pearce',
  'Ben Sharpsteen'],
 'Produced by': 'Walt Disney',
 'Written by': ['Ted Sears',
  'Richard Creedon',
  'Otto Englander',
  'Dick Rickard',
  'Earl Hurd',
  'Merrill De Maris',
  'Dorothy Ann Blank',
  'Webb Smith'],
 'Based on': ['Snow White', 'by The', 'Brothers Grimm'],
 'Starring': ['Adriana Caselotti',
  'Lucille La Verne',
  'Harry Stockwell',
  'Roy Atwell',
  'Pinto Colvig',
  'Otis Harlan',
  'Scotty Mattraw',
  'Billy Gilbert',
  'Eddie Collins',
  'Moroni Olsen',
  'Stuart Buchanan'],
 'Music by': ['Frank Churchill', 'Paul Smith', 'Leigh Harline'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA )',
  'February 4, 1938 (United States)'],
 'Running time': '83 minutes',
 'Country': 'United 

In [28]:
#Convert dates to datetime format
for movie in movie_info_list:
    print([movie.get('Release date','N/A')])

[['May 19, 1937']]
[['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA )', 'February 4, 1938 (United States)']]
[['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)']]
[['November 13, 1940']]
[['June 20, 1941']]
[['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)']]
[['August 9, 1942 (World Premiere-London)', 'August 13, 1942 (Premiere-New York City)', 'August 21, 1942 (U.S.)']]
[['August 24, 1942 (World Premiere-Rio de Janeiro)', 'February 6, 1943 (U.S. Premiere-Boston)', 'February 19, 1943 (U.S.)']]
[['July 17, 1943']]
[['December 21, 1944 (Mexico City)', 'February 3, 1945 (US)']]
[['April 20, 1946 (New York City premiere)', 'August 15, 1946 (U.S.)']]
[['November 12, 1946 (Premiere: Atlanta, Georgia)', 'November 20, 1946']]
[['September 27, 1947']]
['May 27, 1948']
[['November 29, 1948 (Chicago, Illinois)', 'January 19, 1949 (Indianapolis, Indiana)']]
[['October 5, 1949']]
[['February 15, 1950 (Boston)', 'March 4, 1950 (United States)']]

In [29]:
# June 28,1950
from datetime import datetime

dates = [movie.get('Release date','N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date,list):
        date = date[0]

    if date == "N/A":
        return None
    
    date_str = clean_date(date)
   
    formats = ["%B %d, %Y","%d %B %Y"]
    for format in formats:
        try:
            return datetime.strptime(date_str,format)
        except:
            pass
    return None

    

In [30]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date','N/A'))

movie_info_list[1]


{'Title': 'Snow White and the Seven Dwarfs',
 'Directed by': ['David Hand (supervising)',
  'William Cottrell',
  'Wilfred Jackson',
  'Larry Morey',
  'Perce Pearce',
  'Ben Sharpsteen'],
 'Produced by': 'Walt Disney',
 'Written by': ['Ted Sears',
  'Richard Creedon',
  'Otto Englander',
  'Dick Rickard',
  'Earl Hurd',
  'Merrill De Maris',
  'Dorothy Ann Blank',
  'Webb Smith'],
 'Based on': ['Snow White', 'by The', 'Brothers Grimm'],
 'Starring': ['Adriana Caselotti',
  'Lucille La Verne',
  'Harry Stockwell',
  'Roy Atwell',
  'Pinto Colvig',
  'Otis Harlan',
  'Scotty Mattraw',
  'Billy Gilbert',
  'Eddie Collins',
  'Moroni Olsen',
  'Stuart Buchanan'],
 'Music by': ['Frank Churchill', 'Paul Smith', 'Leigh Harline'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA )',
  'February 4, 1938 (United States)'],
 'Running time': '83 minutes',
 'Country': 'United 

In [31]:
import pickle
def save_data_pickle(name,data):
    with open('disney_movie_data-cleaned.pickle','wb') as f:
        pickle.dump(data,f)


In [32]:
def load_data_pickle(name):
    with open(name,'rb') as f:
        return pickle.load(f)

In [33]:
save_data_pickle('disney_movie_data-cleaned.pickle',movie_info_list)

In [34]:
#a = load_data_pickle('disney_movie_data-cleaned.pickle')
movie_info_list = load_data_pickle('disney_movie_data-cleaned.pickle')

### Task 4 : Save data s JSON & CSV

In [35]:
 movie_info_copy = [movie.copy() for movie in movie_info_list] 

In [36]:
 for movie in movie_info_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None


In [1]:
import pandas as pd
df = pd.DataFrame(movie_info_list)
df.to_csv('disney_movie_data_final.csv')

NameError: name 'movie_info_list' is not defined

In [42]:
df.head(5)

,Title,Production company,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),...,Production companies,Japanese,Hepburn,Adaptation by,Animation by,Traditional,Simplified,Created by,Original work,Owned by
0,Academy Award Review of,Walt Disney Productions,"[May 19, 1937]",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,4.547200e+01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,"[December 21, 1937 ( Carthay Circle Theatre , ...",83 minutes,United States,English,$418 million,83.0,1490000.0,4.180000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,Walt Disney Productions,"[February 7, 1940 ( Center Theatre ), February...",88 minutes,United States,English,$164 million,88.0,2600000.0,1.640000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,Walt Disney Productions,"[November 13, 1940]",126 minutes,United States,English,$76.4–$83.3 million,126.0,2280000.0,8.330000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,Walt Disney Productions,"[June 20, 1941]",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,600000.0,9.600000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
